In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("./..")
sys.path.append(".")

In [ ]:
# default libraries
import time
import datetime
import pickle
import pprint
from pathlib import Path
import math


# third party libraries
import torch
import torchvision
import torchvision.transforms as T
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.tensorboard import SummaryWriter

import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from dotted_dict import DottedDict

# local imports
from effcn.models_multimnist import MultiMnistEffCapsNet
from effcn.functions import create_margin_loss
from effcn.utils import count_parameters
from datasets.multimnist import MultiMNist
from misc.optimizer import get_optimizer, get_scheduler


In [ ]:
a = [1,2,3]

x,y,z = a

x

In [ ]:
ds_train = MultiMNist(root='/mnt/data/datasets/multimnist_10',train=True, transform=None, generate=False)
ds_valid = MultiMNist(root='/mnt/data/datasets/multimnist_10',train=False, transform=None, generate=False)



In [ ]:
dl_train = torch.utils.data.DataLoader(ds_train, 
                                    batch_size=8, 
                                    shuffle=False)

In [ ]:
x,y,z,a,b = next(iter(dl_train))

In [ ]:
#x,y,z,a,b =  ds_train[0]
#x2,y2,z2,a2,b2 =  ds_valid[0]

In [ ]:
print(x.shape)
print(y.shape)
print(z.shape)
print(a.shape)
print(b.shape)

In [ ]:
"""
# plot train imgs
x, y, z, a, b = next(iter(ds_train))
img = torchvision.utils.make_grid([x,a,b], nrow=8)
img = img.permute((1,2,0))
plt.imshow(img)
plt.show()
"""

In [ ]:

y_ticks = ["x", 'a', 'b']
x_ticks = torch.transpose(torch.cat((torch.unsqueeze(y,dim=0), torch.unsqueeze(z,dim=0)),dim=0), 0, 1).tolist()


img = torchvision.utils.make_grid(torch.cat([x,a,b], dim=0), nrow=x.shape[0])

#torch.cat([x,a,b], dim=0), nrow=x.shape[0])



#img = img.permute((1,2,0))

y_ticks = ["x", 'a', 'b']
x_ticks = torch.transpose(torch.cat((torch.unsqueeze(y,dim=0), torch.unsqueeze(z,dim=0)),dim=0), 0, 1).tolist()

fig, ax = plt.subplots()

ax.imshow(img.permute((1,2,0)))
ax.set_yticks((np.arange(len(y_ticks))*(x.shape[-1]+2)+(x.shape[-1]/2)), labels=y_ticks)
ax.set_xticks((np.arange(len(x_ticks))*(x.shape[-1]+2)+(x.shape[-1]/2)), labels=x_ticks)
fig.tight_layout()



#fig.savefig("/mnt/data/experiments/EfficientCN/multimnist_overfit/img_valid_xx.png")
#plt.close()

In [ ]:
torch.transpose(torch.cat((torch.unsqueeze(y,dim=0), torch.unsqueeze(z,dim=0)),dim=0), 0, 1).tolist()


In [ ]:
x.shape[1]

In [ ]:
np.arange(len(y_ticks))*x.shape[1]

In [ ]:
np.arange(len(x_ticks))*x.shape[-1]+(x.shape[-1]/2)

In [ ]:
x.shape[-1]

In [ ]:
print(torch.cat([x,a,b], dim=0).shape)

print(img.shape)

In [ ]:
model = MultiMnistEffCapsNet()

model.decoder1

In [ ]:
'from effcn.layers import View, Squash

In [ ]:

class Squash(nn.Module):
    def __init__(self, eps=1e-21):
        super().__init__()
        self.eps = eps

    def forward(self, x):
        """
         IN:  (b, n, d)
         OUT: squash(x(b,n,d))
        """
        x_norm = torch.norm(x, dim=2, keepdim=True)
        #return  (x / (x_norm + self.eps))
        return (1 - 1 / (torch.exp(x_norm) + self.eps)) 
        #return (1 - 1 / (torch.exp(x_norm) + self.eps)) #* (x / (x_norm + self.eps))


In [ ]:
B = Squash()
f = torch.tensor([[[1,1,0,0],[1e-21,1e-21,1e-21,1e-21]]])
B(f).sum(dim=-1)